In [280]:
!pip install torchmetrics

In [281]:
!pip install ipdb

In [282]:
import pandas as pd
import numpy as np
from string import punctuation
from collections import Counter
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import matplotlib.pyplot as plt

import re
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence
from gensim.models import FastText
import torch.optim as optim
import ipdb
from torchmetrics.functional import f1, recall


# Данные

In [283]:
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv
!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv 

In [284]:
pos_tweets = pd.read_csv('../input/nlp-tweets/positive.csv', encoding='utf-8', sep=';', header=None,  names=[0,1,2,'text','tone',5,6,7,8,9,10,11])
neg_tweets = pd.read_csv('../input/nlp-tweets/negative.csv', encoding='utf-8', sep=';', header=None, names=[0,1,2,'text','tone',5,6,7,8,9,10,11] )
neg_tweets['tone'] = 0
all_tweets_data = pos_tweets.append(neg_tweets)
print(len(all_tweets_data))

In [285]:
all_tweets_data

In [286]:
all_tweets_data = shuffle(all_tweets_data[['text','tone']])[:100000]

In [287]:
def preprocess(text):
    text = text.lower().replace("ё", "е")
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', text)
    text = re.sub('@[^\s]+', 'USER', text)
    text = re.sub('[^a-zA-Zа-яА-Я1-9]+', ' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip()

In [288]:
clean_text = all_tweets_data['text'].apply(preprocess)
all_tweets_data['clean_text'] = clean_text

In [289]:
all_tweets_data

In [290]:
train_sentences, val_sentences = train_test_split(all_tweets_data, test_size=0.1)

In [291]:
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
DEVICE

# I

## Датасет

In [292]:
vocab = Counter()

for text in all_tweets_data['clean_text']:
    vocab.update(preprocess(text).split())
print('всего уникальных токенов:', len(vocab))

In [293]:
filtered_vocab = set()

for word in vocab:
    if vocab[word] > 2:
        filtered_vocab.add(word)
print('уникальных токенов, втретившихся больше 2 раз:', len(filtered_vocab))

In [294]:
word2id = {'PAD':0}

for word in filtered_vocab:
    word2id[word] = len(word2id)

id2word = {i:word for word, i in word2id.items()}

In [295]:
def preprocess(text):
        text = text.lower().replace("ё", "е")
        text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', text)
        text = re.sub('@[^\s]+', 'USER', text)
        text = re.sub('[^a-zA-Zа-яА-Я1-9]+', ' ', text)
        text = re.sub(' +', ' ', text)
        return text.strip()

In [296]:
preprocess('";аау"')

In [297]:
class TweetsDataset(Dataset):

    def __init__(self, dataset, word2id, DEVICE):
        self.dataset = dataset['text'].values
        self.word2id = word2id
        self.length = dataset.shape[0]
        self.target = dataset['tone'].values
        self.device = DEVICE

    def __len__(self): #это обязательный метод, он должен уметь считать длину датасета
        return self.length

    def __getitem__(self, index): #еще один обязательный метод. По индексу возвращает элемент выборки
        tokens = self.preprocess(self.dataset[index]) # токенизируем
        ids = torch.LongTensor([self.word2id[token] for token in tokens if token in self.word2id])
        y = [self.target[index]]
        return ids, y
    
    def preprocess(self, text):
        text = text.lower().replace("ё", "е")
        text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', text)
        text = re.sub('@[^\s]+', 'USER', text)
        text = re.sub('[^a-zA-Zа-яА-Я1-9]+', ' ', text)
        text = re.sub(' +', ' ', text)
        return text.strip()

    def collate_fn(self, batch): #этот метод можно реализовывать и отдельно,
    # он понадобится для DataLoader во время итерации по батчам
      ids, y = list(zip(*batch))
      padded_ids = pad_sequence(ids, batch_first=True).to(self.device)
      #мы хотим применять BCELoss, он будет брать на вход predicted размера batch_size x 1 (так как для каждого семпла модель будет отдавать одно число), target размера batch_size x 1
      y = torch.Tensor(y).to(self.device) # tuple ([1], [0], [1])  -> Tensor [[1.], [0.], [1.]] 
      return padded_ids, y

In [298]:
train_dataset = TweetsDataset(train_sentences, word2id, DEVICE)
train_sampler = RandomSampler(train_dataset)
train_iterator = DataLoader(train_dataset, collate_fn = train_dataset.collate_fn, sampler=train_sampler, batch_size=1024)
batch = next(iter(train_iterator))
batch[0].shape

In [299]:
val_dataset = TweetsDataset(val_sentences, word2id, DEVICE)
val_sampler = SequentialSampler(val_dataset)
val_iterator = DataLoader(val_dataset, collate_fn = val_dataset.collate_fn, sampler=val_sampler, batch_size=1024)

## Модель

In [300]:
ft = FastText(all_tweets_data['clean_text'].tolist(), vector_size=100, window=5, min_count=1)
weights = np.zeros((len(word2id), 100))
count = 0
for word, i in word2id.items():
    if word == 'PAD':
        continue   
    try:
        weights[i] = ft.wv[word]    
    except KeyError:
      count += 1
      # oov словам сопоставляем случайный вектор
      weights[i] = np.random.normal(0,0.1,100)

In [301]:
weights = np.zeros((len(word2id), 100))
count = 0
for word, i in word2id.items():
    if word == 'PAD':
        continue   
    try:
        weights[i] = ft.wv[word]    
    except KeyError:
      count += 1
      # oov словам сопоставляем случайный вектор
      weights[i] = np.random.normal(0,0.1,100)

In [302]:
class CNN(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.embedding.from_pretrained(torch.tensor(weights), freeze=True)

        self.bigrams = nn.Conv1d(in_channels=embedding_dim, out_channels=100, kernel_size=2, padding='same')
        self.trigrams = nn.Conv1d(in_channels=embedding_dim, out_channels=80, kernel_size=3, padding='same')
        self.bigrams_over = nn.Conv1d(in_channels=180, out_channels=180, kernel_size=2, padding='same')

        self.pooling = nn.MaxPool1d(kernel_size=2, stride=2)
        self.relu = nn.ReLU()
        self.hidden = nn.Linear(in_features=180, out_features=1)
        self.out = nn.Sigmoid()

    def forward(self, word):
        #batch_size x seq_len
        embedded = self.embedding(word)
        #batch_size x seq_len x embedding_dim
        embedded = embedded.transpose(1,2)
        #batch_size x embedding_dim x seq_len
        feature_map_bigrams = self.relu(self.bigrams(embedded))
        #batch_size x filter_count2 x seq_len* 
        feature_map_trigrams = self.relu(self.trigrams(embedded))
        #batch_size x filter_count3 x seq_len*

        concat = torch.cat((feature_map_bigrams, feature_map_trigrams), 1)
        bigrams = self.pooling(self.relu(self.bigrams_over(concat)))
        pooling = bigrams.max(2)[0] 
        # batch _size x (filter_count2 + filter_count3)

        logits = self.hidden(pooling) 
        logits = self.out(logits)      
        return logits

## Train loop

In [303]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0 # для подсчета среднего лосса на всех батчах

    model.train()  # ставим модель в обучение, явно указываем, что сейчас надо будет хранить градиенты у всех весов

    for i, (texts, ys) in enumerate(iterator): #итерируемся по батчам
        optimizer.zero_grad()  #обнуляем градиенты
        preds = model(texts)  #прогоняем данные через модель
        loss = criterion(preds, ys) #считаем значение функции потерь  
        loss.backward() #считаем градиенты  
        optimizer.step() #обновляем веса 
        epoch_loss += loss.item() #сохраняем значение функции потерь
        if not (i + 1) % int(len(iterator)/5):
            print(f'Train loss: {epoch_loss/i}')      
    return  epoch_loss / len(iterator) # возвращаем среднее значение лосса по всей выборке

In [304]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_metric = 0
    model.eval() 
    with torch.no_grad():
        for i, (texts, ys) in enumerate(iterator):   
            preds = model(texts)  # делаем предсказания на тесте
            loss = criterion(preds, ys)   # считаем значения функции ошибки для статистики  
            epoch_loss += loss.item()
            batch_metric = f1(preds.round().long(), ys.long(), ignore_index=0)
            epoch_metric += batch_metric

            if not (i + 1) % int(len(iterator)/5):
              print(f'Val loss: {epoch_loss/i}, Val f1: {epoch_metric/i}')
        
    return epoch_metric / len(iterator), epoch_loss / len(iterator) # возвращаем среднее значение по всей выборке

## Обучение

Попробуем для начала стандартные настройки из семинара

In [305]:
model = CNN(len(word2id), 2)
optimizer = optim.Adam(model.parameters(), lr=0.0005)
criterion = nn.BCELoss()  

# веса модели и значения лосса храним там же, где и все остальные тензоры
model = model.to(DEVICE)
criterion = criterion.to(DEVICE)

In [306]:
losses = []
losses_eval = []
f1s = []
f1s_eval = []

for i in range(20):
    print(f'\nstarting Epoch {i}')
    print('Training...')
    epoch_loss = train(model, train_iterator, optimizer, criterion)
    losses.append(epoch_loss)
    print('\nEvaluating on train...')
    f1_on_train,_ = evaluate(model, train_iterator, criterion)
    f1s.append(f1_on_train)
    print('\nEvaluating on test...')
    f1_on_test, epoch_loss_on_test = evaluate(model, val_iterator, criterion)
    losses_eval.append(epoch_loss_on_test)
    f1s_eval.append(f1_on_test)

In [307]:
fig, ax = plt.subplots(figsize=(6, 6))
plt.title('Train')
plt.xlabel('Iterations')
plt.ylabel('Losses')
plt.grid()
ax.plot(losses, label='Trainig loss')
ax.plot(losses_eval, label='Evaluation loss')
ax.legend()
plt.show()

In [308]:
fig, ax = plt.subplots(figsize=(6, 6))
plt.title('Train')
plt.xlabel('Iterations')
plt.ylabel('Losses')
plt.grid()
ax.plot(f1s, label='Train F1')
ax.plot(f1s_eval, label='Evaluation F1')
ax.legend()
plt.show()

In [309]:
print('F1 training score: ', f1s[-1])
print('F1 evaluation score: ', f1s_eval[-1])
print('Training loss: ',losses[-1])
print('Evaluation loss: ',losses_eval[-1])

In [310]:
def predict(model, iterator):
    preds = []
    model.eval()
    with torch.no_grad():
        for i, (words, ys) in enumerate(iterator): 
            for word in model(words):
                preds.append(word.cpu().detach().numpy().round())  # делаем предсказания на тесте 
    return preds

In [311]:
pd.set_option('display.max_colwidth', None)

In [312]:
val_sentences['predicted']  = predict(model, val_iterator)

In [313]:
# TP
val_sentences[(val_sentences['tone'] == 1) & (val_sentences['predicted'] == 1)][['clean_text']]

In [314]:
# FN
val_sentences[(val_sentences['tone'] == 1) & (val_sentences['predicted'] == 0)][['clean_text']]

In [315]:
# FP
val_sentences[(val_sentences['tone'] == 0) & (val_sentences['predicted'] == 1)][['clean_text']]

Результаты страннные, но можно заметить что в FN чаще есть сообщения со словом "не" или плохими словами
в FP не распознается сарказм (но это большая придирка, это делать сложно) USER меня ща все заанфолловят наверн

Так же: почему именно последние дни всегда самые лучшие - тожже FP, видимо из-за слова "лучший"

## Улучшение

В архитектуру нейросети добавим dropout=0.5

In [317]:
class CNNDropout(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.embedding.from_pretrained(torch.tensor(weights), freeze=True)

        self.bigrams = nn.Conv1d(in_channels=embedding_dim, out_channels=100, kernel_size=2, padding='same')
        self.trigrams = nn.Conv1d(in_channels=embedding_dim, out_channels=80, kernel_size=3, padding='same')
        self.bigrams_over = nn.Conv1d(in_channels=180, out_channels=180, kernel_size=2, padding='same')

        self.pooling = nn.MaxPool1d(kernel_size=2, stride=2)
        self.relu = nn.ReLU()
        self.hidden = nn.Linear(in_features=180, out_features=1)
        self.dropout = nn.Dropout(p=0.5)
        self.out = nn.Sigmoid()

    def forward(self, word):
        #batch_size x seq_len
        embedded = self.embedding(word)
        #batch_size x seq_len x embedding_dim
        embedded = embedded.transpose(1,2)
        #batch_size x embedding_dim x seq_len
        feature_map_bigrams = self.relu(self.bigrams(embedded))
        #batch_size x filter_count2 x seq_len* 
        feature_map_trigrams = self.relu(self.trigrams(embedded))
        #batch_size x filter_count3 x seq_len*

        concat = torch.cat((feature_map_bigrams, feature_map_trigrams), 1)
        bigrams = self.dropout(self.pooling(self.relu(self.bigrams_over(concat))))
        pooling = bigrams.max(2)[0] 
        # batch _size x (filter_count2 + filter_count3)

        logits = self.hidden(pooling)
        logits = self.out(logits)      
        return logits

Попробуем поднять размерность эмбеддингов до 30, увеличить частоту обучения и добавить L2 регуляризацию
Стандартное значение 1e-2, но я уменьшил его до 1е-4 чтобы не так сильно штрафовать веса
Также увеличим количество эпох

In [318]:
model = CNNDropout(len(word2id), 30)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
criterion = nn.BCELoss()  

# веса модели и значения лосса храним там же, где и все остальные тензоры
model = model.to(DEVICE)
criterion = criterion.to(DEVICE)

In [319]:
losses = []
losses_eval = []
f1s = []
f1s_eval = []

for i in range(20):
    print(f'\nstarting Epoch {i}')
    print('Training...')
    epoch_loss = train(model, train_iterator, optimizer, criterion)
    losses.append(epoch_loss)
    print('\nEvaluating on train...')
    f1_on_train,_ = evaluate(model, train_iterator, criterion)
    f1s.append(f1_on_train)
    print('\nEvaluating on test...')
    f1_on_test, epoch_loss_on_test = evaluate(model, val_iterator, criterion)
    losses_eval.append(epoch_loss_on_test)
    f1s_eval.append(f1_on_test)

In [320]:
fig, ax = plt.subplots(figsize=(6, 6))
plt.title('Train')
plt.xlabel('Iterations')
plt.ylabel('Losses')
plt.grid()
ax.plot(losses, label='Trainig loss')
ax.plot(losses_eval, label='Evaluation loss')
ax.legend()
plt.show()

In [321]:
fig, ax = plt.subplots(figsize=(6, 6))
plt.title('Train')
plt.xlabel('Iterations')
plt.ylabel('Losses')
plt.grid()
ax.plot(f1s, label='Train F1')
ax.plot(f1s_eval, label='Evaluation F1')
ax.legend()
plt.show()

In [322]:
print('F1 training score: ', f1s[-1])
print('F1 evaluation score: ', f1s_eval[-1])
print('Training loss: ',losses[-1])
print('Evaluation loss: ',losses_eval[-1])

Метрики улучшились, посмотрим теперь примеры

In [323]:
val_sentences['predicted']  = predict(model, val_iterator)

In [324]:
# TP
val_sentences[(val_sentences['tone'] == 1) & (val_sentences['predicted'] == 1)][['clean_text']]

In [325]:
# FN
val_sentences[(val_sentences['tone'] == 1) & (val_sentences['predicted'] == 0)][['clean_text']]

In [326]:
# FP
val_sentences[(val_sentences['tone'] == 0) & (val_sentences['predicted'] == 1)][['clean_text']]

Результаты не сильно изменились, но метрики тожже не сильно улучшились

# II

## Датасет

In [327]:
vocab = Counter()

for text in all_tweets_data['clean_text']:
    vocab.update(preprocess(text).split())
print('всего уникальных токенов:', len(vocab))

In [328]:
filtered_vocab = set()

for word in vocab:
    if vocab[word] > 2:
        filtered_vocab.add(word)
print('уникальных токенов, втретившихся больше 2 раз:', len(filtered_vocab))

In [329]:
word2id = {'PAD':0}

for word in filtered_vocab:
    word2id[word] = len(word2id)

id2word = {i:word for word, i in word2id.items()}

In [330]:
symbol_vocab = Counter()
for text in all_tweets_data['clean_text']:
    symbol_vocab.update(list(text))
print('всего уникальных символов:', len(symbol_vocab))

In [331]:
symbol2id = {'PAD':0}

for symbol in symbol_vocab:
    symbol2id[symbol] = len(symbol2id)

id2symbol = {i:symbol for symbol, i in symbol2id.items()}

Добавим символы в датасет

In [332]:
class TweetsDatasetWordSymbol(Dataset):

    def __init__(self, dataset, word2id, symbol2id, DEVICE):
        self.dataset = dataset['clean_text'].values
        self.word2id = word2id
        self.symbol2id = symbol2id

        self.length = dataset.shape[0]
        self.target = torch.Tensor(dataset['tone'].values)
        self.device = DEVICE

    def __len__(self): #это обязательный метод, он должен уметь считать длину датасета
        return self.length

    def __getitem__(self, index): #еще один обязательный метод. По индексу возвращает элемент выборки
        symbols = list(self.dataset[index])
        symbol_ids = torch.LongTensor([self.symbol2id[symbol] for symbol in symbols if symbol in self.symbol2id])
        tokens = self.dataset[index].split()
        word_ids = torch.LongTensor([self.word2id[token] for token in tokens if token in self.word2id])
        y = [self.target[index]]
        return word_ids, symbol_ids, y

    def collate_fn(self, batch): #этот метод можно реализовывать и отдельно,
    # он понадобится для DataLoader во время итерации по батчам
      word_ids, symbol_ids, y = list(zip(*batch))
      padded_words = pad_sequence(word_ids, batch_first=True).to(self.device)
      padded_symbols = pad_sequence(symbol_ids, batch_first=True).to(self.device)
      #мы хотим применять BCELoss, он будет брать на вход predicted размера batch_size x 1 (так как для каждого семпла модель будет отдавать одно число), target размера batch_size x 1 
      y = torch.Tensor(y).to(self.device) # tuple ([1], [0], [1])  -> Tensor [[1.], [0.], [1.]]
      return padded_words, padded_symbols, y

In [333]:
train_dataset = TweetsDatasetWordSymbol(train_sentences, word2id, symbol2id, DEVICE)
train_sampler = RandomSampler(train_dataset)
train_iterator = DataLoader(train_dataset, collate_fn = train_dataset.collate_fn, sampler=train_sampler, batch_size=1024)
batch = next(iter(train_iterator))
batch[0].shape

In [334]:
val_dataset = TweetsDatasetWordSymbol(val_sentences, word2id, symbol2id, DEVICE)
val_sampler = SequentialSampler(val_dataset)
val_iterator = DataLoader(val_dataset, collate_fn = val_dataset.collate_fn, sampler=val_sampler, batch_size=1024)

## Модель

Сразу добавим дропаут в архитектуру

In [335]:
class CNNWordSymbol(nn.Module):
    def __init__(self, word_vocab_size, symbol_vocab_size, symbol_emb_dim):
        super().__init__()

        self.word_embedding = nn.Embedding(word_vocab_size, 100)
        self.word_embedding.from_pretrained(torch.tensor(weights), freeze=True)
        self.symbol_embedding = nn.Embedding(symbol_vocab_size, symbol_emb_dim)
        self.symbol_bigrams = nn.Conv1d(in_channels=symbol_emb_dim, out_channels=100, kernel_size=2, padding='same')
        self.symbol_trigrams = nn.Conv1d(in_channels=symbol_emb_dim, out_channels=80, kernel_size=3, padding='same')
        self.symbol_pooling = nn.MaxPool1d(kernel_size=2, stride=2)
        self.word_hidden = nn.Linear(100, 100)
        self.symbol_hidden = nn.Linear(in_features=180, out_features=100)
        self.linear = nn.Linear(in_features=200, out_features=1)
        self.relu = nn.ReLU()   
        self.dropout = nn.Dropout(p=0.5)
        self.out = nn.Sigmoid()


    def forward(self, word_seq, symbol_seq):
        #batch_size x seq_len
        embedded = self.symbol_embedding(symbol_seq)
        #batch_size x seq_len x embedding_dim
        embedded = embedded.transpose(1,2)
        #batch_size x embedding_dim x seq_len
        feature_map_bigrams = self.symbol_pooling(self.relu(self.symbol_bigrams(embedded)))
        #batch_size x filter_count2 x seq_len* 
        feature_map_trigrams = self.symbol_pooling(self.relu(self.symbol_trigrams(embedded)))
        #batch_size x filter_count3 x seq_len*

        pooling1 = feature_map_bigrams.max(2)[0] 
        # batch_size x filter_count2
        pooling2 = feature_map_trigrams.max(2)[0]
        # batch_size x filter_count3
        concat = torch.cat((pooling1, pooling2), 1)
        # batch_size x (filter_count2 + filter_count3)
        symbol_emb = self.symbol_hidden(concat)

        embedded_words = self.word_embedding(word_seq)
        mean_emb_words = torch.mean(embedded_words, dim=1)
        X = self.dropout(self.word_hidden(mean_emb_words)) 
        X = self.dropout(self.relu(X))
        concat = torch.cat((symbol_emb, X), 1)
        logits = self.out(self.linear(concat))      
        return logits

## Train loop

Сделаем новые обучалки, чтобы учитывать символы

In [336]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0 # для подсчета среднего лосса на всех батчах

    model.train()  # ставим модель в обучение, явно указываем, что сейчас надо будет хранить градиенты у всех весов

    for i, (texts, symbols, ys) in enumerate(iterator): #итерируемся по батчам
        optimizer.zero_grad()  #обнуляем градиенты
        preds = model(texts, symbols)  #прогоняем данные через модель
        loss = criterion(preds, ys) #считаем значение функции потерь  
        loss.backward() #считаем градиенты  
        optimizer.step() #обновляем веса 
        epoch_loss += loss.item() #сохраняем значение функции потерь
        if not (i + 1) % int(len(iterator)/5):
            print(f'Train loss: {epoch_loss/i}')      
    return  epoch_loss / len(iterator) # возвращаем среднее значение лосса по всей выборке

In [337]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_metric = 0
    model.eval() 
    with torch.no_grad():
        for i, (texts, symbols, ys) in enumerate(iterator):   
            preds = model(texts, symbols)  # делаем предсказания на тесте
            loss = criterion(preds, ys)   # считаем значения функции ошибки для статистики  
            epoch_loss += loss.item()
            batch_metric = f1(preds.round().long(), ys.long(), ignore_index=0)
            epoch_metric += batch_metric

            if not (i + 1) % int(len(iterator)/5):
              print(f'Val loss: {epoch_loss/i}, Val f1: {epoch_metric/i}')
        
    return epoch_metric / len(iterator), epoch_loss / len(iterator) # возвращаем среднее значение по всей выборке

## Обучение

Поставим такую же размерность эмбеддингов, частоту обучения и регуляризацию

In [338]:
model = CNNWordSymbol(len(word2id), len(symbol2id), 14)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
criterion = nn.BCELoss()  

# веса модели и значения лосса храним там же, где и все остальные тензоры
model = model.to(DEVICE)
criterion = criterion.to(DEVICE)

In [340]:
losses = []
losses_eval = []
f1s = []
f1s_eval = []

for i in range(20):
    print(f'\nstarting Epoch {i}')
    print('Training...')
    epoch_loss = train(model, train_iterator, optimizer, criterion)
    losses.append(epoch_loss)
    print('\nEvaluating on train...')
    f1_on_train,_ = evaluate(model, train_iterator, criterion)
    f1s.append(f1_on_train)
    print('\nEvaluating on test...')
    f1_on_test, epoch_loss_on_test = evaluate(model, val_iterator, criterion)
    losses_eval.append(epoch_loss_on_test)
    f1s_eval.append(f1_on_test)

In [341]:
fig, ax = plt.subplots(figsize=(6, 6))
plt.title('Train')
plt.xlabel('Iterations')
plt.ylabel('Losses')
plt.grid()
ax.plot(losses, label='Trainig loss')
ax.plot(losses_eval, label='Evaluation loss')
ax.legend()
plt.show()

In [342]:
fig, ax = plt.subplots(figsize=(6, 6))
plt.title('Train')
plt.xlabel('Iterations')
plt.ylabel('Losses')
plt.grid()
ax.plot(f1s, label='Train F1')
ax.plot(f1s_eval, label='Evaluation F1')
ax.legend()
plt.show()

In [343]:
print('F1 training score: ', f1s[-1])
print('F1 evaluation score: ', f1s_eval[-1])
print('Training loss: ',losses[-1])
print('Evaluation loss: ',losses_eval[-1])

In [348]:
def predict_word_symbol(model, iterator):
    preds = []
    model.eval()
    with torch.no_grad():
        for i, (words, symbols, ys) in enumerate(iterator): 
            for word in model(words, symbols):
                preds.append(word.cpu().detach().numpy().round())  # делаем предсказания на тесте 
    return preds

In [349]:
val_sentences['predicted']  = predict_word_symbol(model, val_iterator)

In [350]:
# TP
val_sentences[(val_sentences['tone'] == 1) & (val_sentences['predicted'] == 1)][['clean_text']]

In [351]:
# FN
val_sentences[(val_sentences['tone'] == 1) & (val_sentences['predicted'] == 0)][['clean_text']]

In [352]:
# FP
val_sentences[(val_sentences['tone'] == 0) & (val_sentences['predicted'] == 1)][['clean_text']]

По сравнению с предыдущими моделями, сильных измененний в результатах нет, но с другой стороны их нет, потому что мы убираем много символов в предобработке

## Улучшение

Попробуем улучшить модель, не делая предобработку

In [353]:
vocab = Counter()

for text in all_tweets_data['text']:
    vocab.update(text.split())
print('всего уникальных токенов:', len(vocab))

In [354]:
filtered_vocab = set()

for word in vocab:
    if vocab[word] > 2:
        filtered_vocab.add(word)
print('уникальных токенов, втретившихся больше 2 раз:', len(filtered_vocab))

In [355]:
word2id = {'PAD':0}

for word in filtered_vocab:
    word2id[word] = len(word2id)

id2word = {i:word for word, i in word2id.items()}

In [356]:
symbol_vocab = Counter()
for text in all_tweets_data['text']:
    symbol_vocab.update(list(text))
print('всего уникальных символов:', len(symbol_vocab))

In [357]:
symbol2id = {'PAD':0}

for symbol in symbol_vocab:
    symbol2id[symbol] = len(symbol2id)

id2symbol = {i:symbol for symbol, i in symbol2id.items()}

In [358]:
class TweetsDatasetWordSymbolRaw(Dataset):

    def __init__(self, dataset, word2id, symbol2id, DEVICE):
        self.dataset = dataset['text'].values
        self.word2id = word2id
        self.symbol2id = symbol2id

        self.length = dataset.shape[0]
        self.target = torch.Tensor(dataset['tone'].values)
        self.device = DEVICE

    def __len__(self): #это обязательный метод, он должен уметь считать длину датасета
        return self.length

    def __getitem__(self, index): #еще один обязательный метод. По индексу возвращает элемент выборки
        symbols = list(self.dataset[index])
        symbol_ids = torch.LongTensor([self.symbol2id[symbol] for symbol in symbols if symbol in self.symbol2id])
        tokens = self.dataset[index].split()
        word_ids = torch.LongTensor([self.word2id[token] for token in tokens if token in self.word2id])
        y = [self.target[index]]
        return word_ids, symbol_ids, y

    def collate_fn(self, batch): #этот метод можно реализовывать и отдельно,
    # он понадобится для DataLoader во время итерации по батчам
      word_ids, symbol_ids, y = list(zip(*batch))
      padded_words = pad_sequence(word_ids, batch_first=True).to(self.device)
      padded_symbols = pad_sequence(symbol_ids, batch_first=True).to(self.device)
      #мы хотим применять BCELoss, он будет брать на вход predicted размера batch_size x 1 (так как для каждого семпла модель будет отдавать одно число), target размера batch_size x 1 
      y = torch.Tensor(y).to(self.device) # tuple ([1], [0], [1])  -> Tensor [[1.], [0.], [1.]]
      return padded_words, padded_symbols, y

In [359]:
train_dataset = TweetsDatasetWordSymbolRaw(train_sentences, word2id, symbol2id, DEVICE)
train_sampler = RandomSampler(train_dataset)
train_iterator = DataLoader(train_dataset, collate_fn = train_dataset.collate_fn, sampler=train_sampler, batch_size=1024)
batch = next(iter(train_iterator))
batch[0].shape

In [360]:
val_dataset = TweetsDatasetWordSymbolRaw(val_sentences, word2id, symbol2id, DEVICE)
val_sampler = SequentialSampler(val_dataset)
val_iterator = DataLoader(val_dataset, collate_fn = val_dataset.collate_fn, sampler=val_sampler, batch_size=1024)

In [361]:
model = CNNWordSymbol(len(word2id), len(symbol2id), 2)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()  

# веса модели и значения лосса храним там же, где и все остальные тензоры
model = model.to(DEVICE)
criterion = criterion.to(DEVICE)

In [362]:
losses = []
losses_eval = []
f1s = []
f1s_eval = []

for i in range(10):
    print(f'\nstarting Epoch {i}')
    print('Training...')
    epoch_loss = train(model, train_iterator, optimizer, criterion)
    losses.append(epoch_loss)
    print('\nEvaluating on train...')
    f1_on_train,_ = evaluate(model, train_iterator, criterion)
    f1s.append(f1_on_train)
    print('\nEvaluating on test...')
    f1_on_test, epoch_loss_on_test = evaluate(model, val_iterator, criterion)
    losses_eval.append(epoch_loss_on_test)
    f1s_eval.append(f1_on_test)

In [363]:
fig, ax = plt.subplots(figsize=(6, 6))
plt.title('Train')
plt.xlabel('Iterations')
plt.ylabel('Losses')
plt.grid()
ax.plot(losses, label='Trainig loss')
ax.plot(losses_eval, label='Evaluation loss')
ax.legend()
plt.show()

In [364]:
fig, ax = plt.subplots(figsize=(6, 6))
plt.title('Train')
plt.xlabel('Iterations')
plt.ylabel('Losses')
plt.grid()
ax.plot(f1s, label='Train F1')
ax.plot(f1s_eval, label='Evaluation F1')
ax.legend()
plt.show()

In [365]:
print('F1 training score: ', f1s[-1])
print('F1 evaluation score: ', f1s_eval[-1])
print('Training loss: ',losses[-1])
print('Evaluation loss: ',losses_eval[-1])

In [366]:
val_sentences['predicted']  = predict_word_symbol(model, val_iterator)

In [371]:
# TP
val_sentences[(val_sentences['tone'] == 1) & (val_sentences['predicted'] == 1)][['text']]

In [372]:
# FN
val_sentences[(val_sentences['tone'] == 1) & (val_sentences['predicted'] == 0)][['text']]

In [373]:
# FP
val_sentences[(val_sentences['tone'] == 0) & (val_sentences['predicted'] == 1)][['text']]

Стало намного круче, предобработка плохо влияет на качество модели. А происходит это потому, что для классификации используются специальные символы, известные как эмодзи или смайлики. По ним модели проще определять тональность, например:

@KristinaEchelon тоже верно хд\nОни еще и коричневые. Дааа, пускай будут соплями :DDD - **POS** изза :DDD и так далее.

Это также можно проследить в предыдущих моделях, часто смайлик XD пишут как хд, это помогает классифицировать предложение как позитивное